In [1]:

%load_ext autoreload
%autoreload 2

In [65]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')


In [66]:
# Membership Inference attack (LiRA) on embedding models

# Load dataset
from datasets import load_dataset 
ms_marco = load_dataset("microsoft/ms_marco", "v1.1")


In [67]:

def create_dataset_from_ms_marco(ms_marco, 
                                 split='train'):
    passages = []
    for i in range(len(ms_marco[split])):
        for j in ms_marco[split][i]['passages']['passage_text']:
            passages.append(j)
    return passages

train_passages = create_dataset_from_ms_marco(ms_marco, split='train')
holdout_passages = create_dataset_from_ms_marco(ms_marco, split='validation')

In [68]:

from typing import List
class MSMarcoDataset(torch.utils.data.Dataset):
    def __init__(self, passages: List[str]):
        self.passages = passages

    def __len__(self):
        return len(self.passages)

    def __getitem__(self, idx):
        return self.passages[idx]
    
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MSMarcoDataset(train_passages), batch_size=32, shuffle=True)
holdout_dataloader = DataLoader(MSMarcoDataset(holdout_passages), batch_size=32, shuffle=True)


In [69]:

from ml_security.attacks.membership_inference_attack import create_attack_dataloader

def get_confidence_scores(
    model, dataloader, device
):
    confidence_scores = []
    for batch in tqdm(dataloader):
                
        # Tokenize sentences
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

        confidence_scores.append(sentence_embeddings)

    return torch.cat(confidence_scores)
            


attack_dataloader, attack_labels = create_attack_dataloader(train_dataloader, holdout_dataloader, model, device='cpu', get_confidence_scores=get_confidence_scores)

KeyboardInterrupt: 

In [63]:
# Create an Attacker Model

import torch

class AttackerModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(AttackerModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    

In [64]:
# Initialize the attack model.
device = torch.device("cpu") 
from torch import nn, optim
from tqdm import tqdm
import numpy as np
from ml_security.logger import logger

attack_model = AttackerModel(input_dim=768, hidden_dim=100)
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.001)

# Trains the attack model.
attack_model.train()

for epoch in range(100):
    for data, target in tqdm(attack_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = attack_model(data)
        loss = criterion(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        logger.info("Finished epoch", epoch=epoch, loss=loss.item())

attack_model.eval()

attack_predictions = []
with torch.no_grad():
    for data, target in tqdm(attack_dataloader):
        data, target = data.to(device), target.to(device)
        output = attack_model(data)
        attack_predictions.append(output.cpu().numpy())

attack_predictions = np.concatenate(attack_predictions)

# Calculate the accuracy of the attack model.
attack_accuracy = np.mean((attack_predictions > 0.5) == attack_labels)
logger.info("Attack stats", accuracy=attack_accuracy)

100%|██████████| 19/19 [00:00<00:00, 394.46it/s]

tensor([[-0.0080, -0.1031,  0.0097,  ...,  0.0269, -0.0062,  0.0254],
        [ 0.0404,  0.0258, -0.0073,  ...,  0.0232,  0.0075,  0.0418],
        [ 0.0078,  0.0571, -0.0114,  ..., -0.0054,  0.0161,  0.0421],
        [-0.0363,  0.0295,  0.0134,  ...,  0.0071, -0.0118,  0.0239]])
tensor([0., 0., 0., 1.])
tensor([[0.5134],
        [0.5143],
        [0.5146],
        [0.5139]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[-0.0286,  0.0283, -0.0180,  ..., -0.0301,  0.1082,  0.0145],
        [ 0.0372,  0.0131, -0.0149,  ..., -0.0276,  0.0126,  0.0068],
        [-0.0726,  0.0323,  0.0175,  ...,  0.0505, -0.0096,  0.0314],
        [ 0.0178,  0.0274, -0.0129,  ..., -0.0051, -0.0172,  0.0250]])
tensor([0., 1., 1., 0.])
tensor([[0.5097],
        [0.5120],
        [0.5153],
        [0.5048]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[-0.0044, -0.1072,  0.0145,  ...,  0.0085,  0.0215,  0.0222],
        [ 0.0307,  0.0373, -0.0381,  ..., -0.0184, -0.0297,  0.0301],
        


100%|██████████| 19/19 [00:00<00:00, 549.61it/s]


tensor([[-0.0427,  0.0074,  0.0280,  ...,  0.0173,  0.0065,  0.0202],
        [ 0.0404,  0.0258, -0.0073,  ...,  0.0232,  0.0075,  0.0418],
        [ 0.0456, -0.0335,  0.0115,  ..., -0.0148, -0.0239, -0.0414],
        [-0.0169, -0.1078,  0.0205,  ...,  0.0020,  0.0071,  0.0254]])
tensor([1., 0., 0., 0.])
tensor([[0.5574],
        [0.4340],
        [0.4485],
        [0.4453]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[-0.0037, -0.0073,  0.0002,  ...,  0.0470, -0.0449,  0.0329],
        [-0.0489,  0.0296, -0.0038,  ...,  0.0496, -0.0148,  0.0175],
        [-0.0080, -0.1031,  0.0097,  ...,  0.0269, -0.0062,  0.0254],
        [ 0.0505, -0.0477,  0.0036,  ..., -0.0643, -0.0593, -0.0160]])
tensor([1., 1., 0., 1.])
tensor([[0.5606],
        [0.5739],
        [0.4403],
        [0.5436]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[-0.0168, -0.0998,  0.0203,  ...,  0.0274, -0.0053,  0.0315],
        [-0.0431,  0.0160,  0.0128,  ...,  0.0045, -0.0127,  0.0305],
        

100%|██████████| 19/19 [00:00<00:00, 1084.23it/s]


...

tensor([[1.3721e-04],
        [6.9246e-05],
        [9.9991e-01],
        [9.9990e-01]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])


  0%|          | 0/19 [00:00<?, ?it/s]

tensor([[ 0.0110, -0.0095,  0.0166,  ..., -0.0095, -0.0651,  0.0229],
        [ 0.0372,  0.0131, -0.0149,  ..., -0.0276,  0.0126,  0.0068],
        [-0.0169, -0.1078,  0.0205,  ...,  0.0020,  0.0071,  0.0254],
        [-0.0262, -0.0021, -0.0015,  ..., -0.0118,  0.0826, -0.0077]])
tensor([1., 1., 0., 0.])
tensor([[9.9978e-01],
        [9.9986e-01],
        [6.7504e-05],
        [5.5724e-05]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[-0.0042,  0.0311,  0.0116,  ...,  0.0309, -0.0252,  0.0238],
        [-0.0010,  0.0107, -0.0042,  ..., -0.0083, -0.0120, -0.0084],
        [ 0.0456, -0.0335,  0.0115,  ..., -0.0148, -0.0239, -0.0414],
        [-0.0221, -0.0028, -0.0347,  ..., -0.0233,  0.0124,  0.0116]])
tensor([1., 1., 0., 1.])
tensor([[9.9963e-01],
        [9.9995e-01],
        [9.3318e-05],
        [9.9998e-01]], grad_fn=<SigmoidBackward0>)
torch.Size([4, 1])
tensor([[ 0.0830, -0.0373, -0.0302,  ..., -0.0255, -0.0089,  0.0090],
        [-0.0044, -0.1072,  0.0145,  ...,  0.0

100%|██████████| 19/19 [00:00<00:00, 5736.94it/s]

2024-10-21 14:36:54 [info     ] Attack stats                   accuracy=np.float64(0.5013850415512465)
